# Problema de selección Quantum Mads (CQM)

---
### Contexto
Un determinado puerto marítimo tiene las siguientes ubicaciones:
- 3 zonas de descarga C1, C2 y C3
- 2 zonas de carga S1, S2
- 2 depósitos para almacenaje D1, D2
- 3 terminales T1, T2 y T3. 

Estas, se nombrarán mediante la siguiente convención sin perdida de generalización:
- C1: **0**; C2: **1**; C3: **2**
- D1: **3**; D2: **4**
- T1: **5**; T2: **6**; T3:**7**
- S1: **8**; S2: **9**

Cada instante de tiempo se reciben y solicitan contenedores vacíos, que se redistribuyen para satisfacer la oferta y la demanda de comercio ultramarino. 

---

### Datos 
Las capacidades máximas de almacenaje tanto en terminales como en depósitos son de 20 contenedores.
Los costes de almacenaje por contenedor son:
- 20 para la terminal T1
- 30 para la terminal T2
- 25 para la terminal T3
- 22 para el depósito D1
- 28 para el depósito D2. 

En un cierto instante de tiempo, cada depósito y cada terminal, excepto la T2, tienen un contenedor vacío en stock. Adicionalmente, en ese mismo instante, cada zona de carga y la terminal T2 reclaman cada una un contenedor vacío para importación ultramarítima, así como cada zona de descarga provee de un contenedor vacío proveniente de comercio de ultra mar. 

Los costes de transporte de contenedores vienen dados por la siguiente matriz agrupada, que contemplan los únicos desplazamientos permitidos (que son a su vez asimétricos): 
- El elemento de la matriz: c[o][d] representa el coste de ir del origen (o) al destino (d)

\begin{bmatrix}
0 & 0 & 0 & 12 & 10 & 14 & 15 & 12 & 0 & 0 \\
0 & 0 & 0 & 19 & 12 & 15 & 17 & 12 & 0 & 0 \\
0 & 0 & 0 & 16 & 18 & 20 & 19 & 17 & 0 & 0 \\
0 & 0 & 0 & 0  & 18 & 18 & 16 & 17 & 11 & 17 \\
0 & 0 & 0 & 16 & 0  & 14 & 17 & 18 & 12 & 14 \\
0 & 0 & 0 & 16 & 15 & 0  & 15 & 18 & 12 & 10 \\
0 & 0 & 0 & 14 & 17 & 15 & 0  & 12 & 19 & 12  \\
0 & 0 & 0 & 13 & 16 & 17 & 16 & 0  & 16 & 18  \\
0 & 0 & 0 & 0  & 0  & 0  & 0  & 0  & 0  & 0  \\
0 & 0 & 0 & 0  & 0  & 0  & 0  & 0  & 0  & 0
\end{bmatrix}

---

### Objetivo

Plantear el problema de optimización para reducir costes y satisfacer la oferta y demanda de contenedores vacíos en ese instante. El resultado ideal debería de ser:
- Movimientos de contenedores que cada solución sugiere
- ¿Es la solución capaz de satisfacer la demanda?
- Coste total asociado.

---

In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

#Preparación del escenario
origen = [0,1,2,3,4,5,6,7,8,9] #Ubicación inicial
destino = [0,1,2,3,4,5,6,7,8,9] #Ubicación final
costes_de_trayecto = [
    [0, 0, 0, 12, 10, 14, 15, 12, 0, 0],
    [0, 0, 0, 19, 12, 15, 17, 12, 0, 0],
    [0, 0, 0, 16, 18, 20, 19, 17, 0, 0],
    [0, 0, 0, 0, 18, 18, 16, 17, 11, 17],
    [0, 0, 0, 16, 0, 14, 17, 18, 12, 14],
    [0, 0, 0, 16, 15, 0, 15, 18, 12, 10],
    [0, 0, 0, 14, 17, 15, 0, 12, 19, 12],
    [0, 0, 0, 13, 16, 17, 16, 0, 16, 18],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],]
costes_de_almacenamiento = {3: 22, 4: 28, 5: 20, 6: 30, 7: 25}
costes = costes_de_trayecto
for fila in costes: #La matriz final contiene los costes asociados al transporte y almacenamiento final de cada trayecto
    for columna, valor in costes_de_almacenamiento.items():
        fila[columna] += valor  

#Una variable para cada posible ubicación final de cada ubicación inicial
x = [[f'x{o}{d}' for d in destino] for o in origen]

#Iniciar BQM
bqm=BinaryQuadraticModel('BINARY')

#Objetivo
for o in origen:
    for d in destino:
        bqm.add_variable(x[o][d], costes[o][d]) 

#Restricciones
#a) Capacidad máxima de almacenamiento (TRIVIAL)
destinos_almacenaje = [3,4,5,6,7]
for d in destinos_almacenaje:
    r1 = [(x[o][d],1) for o in origen]
    bqm.add_linear_inequality_constraint(r1,
            ub = 20,
            lagrange_multiplier = 1,
            label = 'r1_capacidad_'+str(d))

#b) Solicitud de contenedores (demanda)
destinos_carga = [8,9]
for d in destinos_carga:
    r2 = [(x[o][d],1) for o in origen]
    bqm.add_linear_equality_constraint(r2,
            constant = 1,
            lagrange_multiplier = 1)
            
destinos_demanda = [6]
for d in destinos_demanda:
    r3 = [(x[o][d],1) for o in origen]
    bqm.add_linear_inequality_constraint(r3,
            lb = 1,
            ub = 20,
            lagrange_multiplier = 1,
            label = 'r3_demanda_'+str(d)) 
    
#c) Provisión de contenedores (oferta)
origen_descarga = [0,1,2]
for o in origen_descarga:
    r4 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r4,
            constant = 1,
            lagrange_multiplier = 1)

#d) Condición inicial
origen_contenedores = [3,4,5,7]
for o in origen_contenedores:
    r5 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r5,
            constant = 1,
            lagrange_multiplier = 1)
    
#e) Prohibición de permanencia en descargas
destinos_descarga = [0,1,2]
for d in destinos_descarga:
    r6 = [(x[o][d],1) for o in origen]
    bqm.add_linear_equality_constraint(r6,
            constant = 0,
            lagrange_multiplier = 1)  

#f) Ubicaciones sin contenedores inicialmente
origen_vacio = [6,8,9]
for o in origen_vacio:
    r7 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r7,
            constant = 0,
            lagrange_multiplier = 1)  
    
#g) Imposibilidad de tránsito Descarga-Carga
for o in origen_descarga:
    r8 = [(x[o][d],1) for d in destinos_carga]
    bqm.add_linear_equality_constraint(r8,
            constant = 0,
            lagrange_multiplier = 1)  

#h) número de trayectos
r9 = [(x[o][d], 1) for o in origen for d in destino] 
bqm.add_linear_equality_constraint(r9,
    constant=7,  
    lagrange_multiplier=1)      
         

C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_3. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_4. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_5. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_6. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python

In [2]:
sampler = EmbeddingComposite(DWaveSampler(token = "DEV-5042821ad2e2d999ab962237ae05315c378993b2"))
sampleset = sampler.sample(bqm, num_reads=100)

sample= sampleset.first.sample
coste_total = 0

print("\n\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9")
for o in origen:
    printout =str(o)
    for d in range (len(destino)):
        printout += "\t" + str(sample[x[o][d]])
        coste_total += sample[x[o][d]]*costes[o][d]
    print (printout)

print("Coste total:\t", coste_total, "\n") 


	0	1	2	3	4	5	6	7	8	9
0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0
2	0	0	0	0	0	0	0	0	0	0
3	0	0	0	0	0	0	0	0	0	0
4	0	0	0	0	0	0	0	0	0	0
5	0	0	0	0	0	0	0	0	0	0
6	0	0	0	0	0	0	0	0	0	0
7	0	0	0	0	0	0	0	0	0	0
8	0	0	0	0	0	0	0	0	0	0
9	0	0	0	0	0	0	0	0	0	0
Coste total:	 0 

